In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd


from pyspark.ml import *
from pyspark.ml.classification import *
from pyspark.ml.feature import *
from pyspark.ml.param import *
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import *
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import rand 
from sklearn.metrics import classification_report
from time import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row

In [10]:
print ("Running Spark Version %s" % (sklearn.version))


NameError: name 'sklearn' is not defined

In [4]:
rdd1 = sc.textFile("amazon_cells_labelled.txt")
rdd1.take(5)

NameError: name 'sc' is not defined